In [43]:
import pandas as pd
# from azure.ai.ml import MLClient
# from azure.identity import DefaultAzureCredential
from sklearn.model_selection import train_test_split

# ml_client = MLClient.from_config(credential=DefaultAzureCredential())
# data_asset = ml_client.data.get("allo_cine", version="1.1")

df = pd.read_parquet("allocine_spider_clean.parquet")
df.head()

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Michelle Pfeiffer, Lucas Hedges, Valerie Maha...",2.1,2021-06-20,Azazel Jacobs,Sony Pictures Releasing France,"[Comédie, Drame]",[Anglais],110.0,"[Canada, Irlande, France]",NaN,NaN,-,French Exit,None,/article/fichearticle_gen_carticle=18706328.html
1,"[Margot Nuccetelli, Dave Willetts, Rick Kavanian]",2.4,2022-07-20,Adam Gunn,Alba Films,"[Animation, Famille, Fantastique]",[Anglais],82.0,"[Allemagne, Australie, Belgique, Inde]",2.5,42006.0,157299,"Mia et moi, L’Héroïne de Centopia",Mia And Me - The Movie,/article/fichearticle_gen_carticle=18711883.html
2,"[Éric Nantchouang, Salif Cissé, Édouard Sulpice]",3.9,2021-07-21,Guillaume Brac,Arte,[Comédie],[Français],95.0,[France],3.7,5612.0,149445,À l’abordage,None,/article/fichearticle_gen_carticle=18706984.html
3,"[Anthony Hopkins, Olivia Colman, Mark Gatiss]",3.9,2021-05-26,Florian Zeller,Orange Studio Distribution / UGC Distribution,[Drame],[Anglais],98.0,"[Grande-Bretagne, France]",4.3,153927.0,151195,The Father,None,/article/fichearticle_gen_carticle=1000021987....
4,"[Josiane Balasko, Didier Bourdon, Marilou Berry]",2.8,2021-12-15,Alexandra Leclère,UGC Distribution,[Comédie],[Français],95.0,[France],2.6,198247.0,-,Mes très chers enfants,None,/article/fichearticle_gen_carticle=18708877.html


In [44]:
df = df[df['french_first_week_boxoffice'].notna()]

In [45]:
df.head()

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
1,"[Margot Nuccetelli, Dave Willetts, Rick Kavanian]",2.4,2022-07-20,Adam Gunn,Alba Films,"[Animation, Famille, Fantastique]",[Anglais],82.0,"[Allemagne, Australie, Belgique, Inde]",2.5,42006.0,157299,"Mia et moi, L’Héroïne de Centopia",Mia And Me - The Movie,/article/fichearticle_gen_carticle=18711883.html
2,"[Éric Nantchouang, Salif Cissé, Édouard Sulpice]",3.9,2021-07-21,Guillaume Brac,Arte,[Comédie],[Français],95.0,[France],3.7,5612.0,149445,À l’abordage,None,/article/fichearticle_gen_carticle=18706984.html
3,"[Anthony Hopkins, Olivia Colman, Mark Gatiss]",3.9,2021-05-26,Florian Zeller,Orange Studio Distribution / UGC Distribution,[Drame],[Anglais],98.0,"[Grande-Bretagne, France]",4.3,153927.0,151195,The Father,None,/article/fichearticle_gen_carticle=1000021987....
4,"[Josiane Balasko, Didier Bourdon, Marilou Berry]",2.8,2021-12-15,Alexandra Leclère,UGC Distribution,[Comédie],[Français],95.0,[France],2.6,198247.0,-,Mes très chers enfants,None,/article/fichearticle_gen_carticle=18708877.html
5,"[Virginie Efira, Albert Dupontel, Nicolas Marié]",3.7,2020-10-21,Albert Dupontel,Gaumont Distribution,[Comédie],[Français],87.0,[France],3.9,600444.0,150859,Adieu Les Cons,None,/video/player_gen_cmedia=19589300&cfilm=274345...


In [ ]:
features_of_interest = [
    'actors',
    'critics_score',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']

numerical_features = ['critics_score', 'length', 'viewers_score']
date_feature = ['date']
categorical_features = ['director', 'editor']
list_categorical_features = ['actors', 'genre', 'langage', 'nationality']


list_categorical_features = ['actors', 'genre', 'langage', 'nationality']
for col in list_categorical_features:
    df[col] = df[col].mask(df[col].isna(), ['no value'])

target = 'french_first_week_boxoffice'

X, y = (
    df[features_of_interest],
    df[target]
)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [47]:
df = df.sort_values('date')
train_size = int(len(df) * 0.9)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]
X_train = train_df[features_of_interest]
y_train = train_df[target]

X_test = test_df[features_of_interest]
y_test = test_df[target]

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import numpy as np

class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlbs = {}  # Stocke un MultiLabelBinarizer pour chaque colonne
    
    def fit(self, X, y=None):
        for col in X.columns:
            self.mlbs[col] = MultiLabelBinarizer()
            self.mlbs[col].fit(X[col])
        return self
    
    def transform(self, X):
        transformed_list = []
        for col in X.columns:
            transformed = self.mlbs[col].transform(X[col])
            new_columns = [f"{col}_{label}" for label in self.mlbs[col].classes_]
            transformed_list.append(pd.DataFrame(transformed, columns=new_columns, index=X.index))
        
        return pd.concat(transformed_list, axis=1)
    
    def get_feature_names_out(self, input_features=None):
        # Collecter tous les noms de colonnes de sortie
        feature_names = []
        for col in self.mlbs.keys():
            feature_names.extend([f"{col}_{label}" for label in self.mlbs[col].classes_])
        return np.array(feature_names)

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Bourin... à changer mais j'ai même pas vérifier s'il manquait des données...
    ('scaler', StandardScaler())
])

class CustomDateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_names_out = ['year', 'month', 'day', 'dayofweek']
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        result = pd.DataFrame({
            'year': X['date'].dt.year,
            'month': X['date'].dt.month,
            'day': X['date'].dt.day,
            'dayofweek': X['date'].dt.dayofweek
        })
        return result
        
    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names_out)

date_transformer = Pipeline(steps=[
    ('date_features', CustomDateTransformer())
])

# Toujours bourin :p
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

list_categorical_transformer = Pipeline(steps=[
    ('multi_label', MultiLabelBinarizerTransformer())
])

# Création du préprocesseur (sans les colonnes déjà transformées)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('date', date_transformer, date_feature),
        ('cat', categorical_transformer, categorical_features),
        ('list', list_categorical_transformer, list_categorical_features)
    ],
    remainder='passthrough'
)

In [54]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
xgboost_model = Pipeline([
    ('preprocessor', preprocessor),  
    ('xgboost', xgb.XGBRegressor())
])

xgboost_model.fit(X_train,y_train)
y_pred = xgboost_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MAPE: {mape} %")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MAE: 84850.20212419865
MAPE: 15.453101799403676 %
RMSE: 238492.8688202864
R²: 0.2919859306394873


/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Filipino'] will be ignored
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [50]:
list_categorical_features = ['genre', 'langage', 'nationality', 'actors']
for col in list_categorical_features:
    df[col] = df[col].mask(df[col].isna(), ['no value'])
    
df['french_prod']= df['nationality'].apply(lambda x: 1 if "France" in x else 0)
df['usa_prod']= df['nationality'].apply(lambda x: 1 if "U.S.A." in x else 0)


list_categorical_features = ['genre', 'langage']
features_of_interest = [
    'french_prod',
    'critics_score',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'usa_prod',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']

numerical_features = ['critics_score', 'length', 'viewers_score']
date_feature = ['date']
categorical_features = ['director', 'editor']
list_categorical_features = ['genre', 'langage']


target = 'french_first_week_boxoffice'

X, y = (
    df[features_of_interest],
    df[target]
)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [51]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('date', date_transformer, date_feature),
        ('cat', categorical_transformer, categorical_features),
        ('list', list_categorical_transformer, list_categorical_features)
    ],
    remainder='passthrough'
)

xgboost_model = Pipeline([
    ('preprocessor', preprocessor),  
    ('xgboost', xgb.XGBRegressor())
])

xgboost_model.fit(X_train,y_train)
y_pred = xgboost_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MAE: 84850.20212419865
RMSE: 238492.8688202864
R²: 0.2919859306394873


/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Filipino'] will be ignored
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
